In [68]:
import datasets

# mlabonne/FineTome-100k

In [2]:
ds_finetome = datasets.load_dataset("mlabonne/FineTome-100k")

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [4]:
ds_finetome=ds_finetome["train"].shuffle(seed=42).select(range(10000))

In [5]:
ds_finetome

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 10000
})

In [7]:
pds = ds_finetome.to_pandas()

In [15]:
pds.groupby(['source']).count()

,conversations,score
source,,
WebInstructSub_axolotl,2357,2357
airoboros-3.2,225,225
financial-instructions-cleaned-2,126,126
glaive-function-calling-v2-sharegpt,320,320
infini-instruct-top-500k,4290,4290
reasoning-sharegpt,336,336
self-instruct-sharegpt,542,542
systemchat-2.0-sharegpt,173,173
ultrainteract_trajectories_sharegpt,1631,1631


In [22]:
ds_finetome[0]["conversations"]

[{'from': 'human', 'value': 'Give three types of computer graphics.'},
 {'from': 'gpt',
  'value': '1. Raster Graphics: These are also called bitmap graphics and are composed of pixels arranged in a grid. Each pixel can have a different color and shade. Raster graphics excel at representing photographic images and digital painting.\n\n2. Vector Graphics: These graphics are constructed using mathematical formulas representing geometric shapes like lines, curves, and polygons. They are resolution-independent, meaning they can be scaled up or down in size without losing quality. Vector graphics are commonly used for logos, icons, typography and illustrations.\n\n3. 3D Graphics: These graphics are used to create three-dimensional digital representations of objects. 3D graphics use techniques like modeling, rendering, and shading to simulate depth and surface properties. These graphics are used in animation, video games, architecture, engineering, and virtual reality.'}]

In [50]:
ds_finetome = ds_finetome.map(lambda x: {"conversations": [{"role":"user" if con['from']=='human' else "assistant", "content":con['value']} for con in x["conversations"]]})

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [51]:
ds_finetome[0]["conversations"]

[{'content': 'Give three types of computer graphics.', 'role': 'user'},
 {'content': '1. Raster Graphics: These are also called bitmap graphics and are composed of pixels arranged in a grid. Each pixel can have a different color and shade. Raster graphics excel at representing photographic images and digital painting.\n\n2. Vector Graphics: These graphics are constructed using mathematical formulas representing geometric shapes like lines, curves, and polygons. They are resolution-independent, meaning they can be scaled up or down in size without losing quality. Vector graphics are commonly used for logos, icons, typography and illustrations.\n\n3. 3D Graphics: These graphics are used to create three-dimensional digital representations of objects. 3D graphics use techniques like modeling, rendering, and shading to simulate depth and surface properties. These graphics are used in animation, video games, architecture, engineering, and virtual reality.',
  'role': 'assistant'}]

In [64]:
ds_finetome = ds_finetome.remove_columns(["source", "score"])

In [65]:
ds_finetome

Dataset({
    features: ['conversations'],
    num_rows: 10000
})

# habanoz/WildChat-turkce

In [23]:
ds_wc_tr = datasets.load_dataset("habanoz/WildChat-turkce")

Generating train split:   0%|          | 0/6104 [00:00<?, ? examples/s]

In [25]:
ds_wc_tr= ds_wc_tr["train"]

In [26]:
ds_wc_tr

Dataset({
    features: ['conversation_hash', 'model', 'timestamp', 'conversation', 'turn', 'language', 'openai_moderation', 'detoxify_moderation', 'toxic', 'redacted', 'state', 'country', 'hashed_ip', 'header'],
    num_rows: 6104
})

In [34]:
ds_wc_tr = ds_wc_tr.remove_columns(['conversation_hash', 'model', 'timestamp', 'turn', 'language', 'openai_moderation', 'detoxify_moderation', 'toxic', 'redacted', 'state', 'country', 'hashed_ip', 'header'])

In [35]:
ds_wc_tr

Dataset({
    features: ['conversation'],
    num_rows: 6104
})

In [44]:
ds_wc_tr = ds_wc_tr.map(lambda x: {"conversations": [{"role":con['role'], "content":con['content']} for con in x["conversation"]]}).remove_columns(['conversation'])

Map:   0%|          | 0/6104 [00:00<?, ? examples/s]

In [48]:
ds_wc_tr[0]['conversations']

[{'content': 'bu metni türkçeye çevir', 'role': 'user'},
 {'content': "bu metni Türkçe'ye çevir", 'role': 'assistant'},
 {'content': "User Consent for Data Collection, Use, and Sharing\n▼\nBy using our app, which is powered by OpenAI's API, you acknowledge and agree to the following terms regarding the data you provide:\n\nCollection: We may collect information, including the inputs you type into our app, the outputs generated by OpenAI's API, and certain technical details about your device and connection (such as browser type, operating system, and IP address) provided by your device's request headers.\nUse: We may use the collected data for research purposes, to improve our services, and to develop new products or services, including commercial applications, and for security purposes, such as protecting against unauthorized access and attacks.\nSharing and Publication: Your data, including the technical details collected from your device's request headers, may be published, shared wi

## Custom

In [55]:
from datasets import Dataset

ds_custom = Dataset.from_dict({"conversations": [
    [{"role": "user", "content": "Sen kimsin?"}, {"role": "assistant", "content": "Benim adım TuristGPT, codetrio tarafından geliştirilen bir yapay zeka asistanıyım. Size nasıl yardımcı olabilirim?"}],
    [{"role": "user", "content": "Benim için neler yapabilirsin?"}, {"role": "assistant", "content": "Bir seyahat planlaması yapmana yardımcı olabilirim. Gideceğin yer hakkında bilgi verebilir, konaklama ve ulaşım seçenekleri sunabilir, gezilecek yerler hakkında önerilerde bulunabilirim."}],
]})

In [56]:
ds_custom

Dataset({
    features: ['conversations'],
    num_rows: 2
})

In [59]:
ds_custom[0]['conversations']

[{'content': 'Sen kimsin?', 'role': 'user'},
 {'content': 'Benim adım TuristGPT, codetrio tarafından geliştirilen bir yapay zeka asistanıyım. Size nasıl yardımcı olabilirim?',
  'role': 'assistant'}]

# Merged Dataset

In [66]:
ds = datasets.concatenate_datasets([ds_finetome, ds_wc_tr, ds_custom])

In [ ]:
import os
os.environ["HF_TOKEN"] = "xxxx"

In [67]:
ds.push_to_hub("habanoz/FineTume10k-WildChatTr6k", private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/habanoz/FineTume10k-WildChatTr6k/commit/1a6791842adda667b245a36e170d3a9c6f457b9e', commit_message='Upload dataset', commit_description='', oid='1a6791842adda667b245a36e170d3a9c6f457b9e', pr_url=None, pr_revision=None, pr_num=None)

# Analyse Data

In [5]:
import datasets

In [6]:
ds_ftwctr = datasets.load_dataset("habanoz/FineTume10k-WildChatTr6k")

In [7]:
ds_ftwctr = ds_ftwctr["train"]

In [8]:
ds_ftwctr

Dataset({
    features: ['conversations'],
    num_rows: 16106
})

In [2]:
from transformers import AutoTokenizer

In [10]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-8B-Base")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [11]:
tokenizer.chat_template

'{%- if tools %}\n    {{- \'<|im_start|>system\\n\' }}\n    {%- if messages[0].role == \'system\' %}\n        {{- messages[0].content + \'\\n\\n\' }}\n    {%- endif %}\n    {{- "# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>" }}\n    {%- for tool in tools %}\n        {{- "\\n" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- "\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\"name\\": <function-name>, \\"arguments\\": <args-json-object>}\\n</tool_call><|im_end|>\\n" }}\n{%- else %}\n    {%- if messages[0].role == \'system\' %}\n        {{- \'<|im_start|>system\\n\' + messages[0].content + \'<|im_end|>\\n\' }}\n    {%- endif %}\n{%- endif %}\n{%- set ns = namespace(multi_step_tool=true, last_query_index=messages|length - 1) %}\n{%- for message in messa

In [14]:
ds_ftwctr = ds_ftwctr.map(lambda x: {"input_ids": tokenizer.apply_chat_template(x["conversations"], tokenize=True)})

Map:   0%|          | 0/16106 [00:00<?, ? examples/s]

In [15]:
ds_ftwctr

Dataset({
    features: ['conversations', 'input_ids'],
    num_rows: 16106
})

In [16]:
tokenizer.apply_chat_template(ds_ftwctr[0]["conversations"], tokenize=False)

'<|im_start|>user\nGive three types of computer graphics.<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n1. Raster Graphics: These are also called bitmap graphics and are composed of pixels arranged in a grid. Each pixel can have a different color and shade. Raster graphics excel at representing photographic images and digital painting.\n\n2. Vector Graphics: These graphics are constructed using mathematical formulas representing geometric shapes like lines, curves, and polygons. They are resolution-independent, meaning they can be scaled up or down in size without losing quality. Vector graphics are commonly used for logos, icons, typography and illustrations.\n\n3. 3D Graphics: These graphics are used to create three-dimensional digital representations of objects. 3D graphics use techniques like modeling, rendering, and shading to simulate depth and surface properties. These graphics are used in animation, video games, architecture, engineering, and virtual reality.<|im_end

In [17]:
ds_ftwctr = ds_ftwctr.map(lambda x: {"len": len(x["input_ids"])})

Map:   0%|          | 0/16106 [00:00<?, ? examples/s]

In [18]:
ds_ftwctr

Dataset({
    features: ['conversations', 'input_ids', 'len'],
    num_rows: 16106
})

In [19]:
dsp = ds_ftwctr.to_pandas()

In [30]:
dsp["len"].quantile([0.5, 0.75, 0.9 , 0.95, 0.99, 0.999, 1.00])

0.500       510.00
0.750       968.00
0.900      1946.00
0.950      3124.00
0.990      7092.90
0.999     31061.09
1.000    110925.00
Name: len, dtype: float64

In [31]:
ds_ftwctr_8192 = ds_ftwctr.filter(lambda x: x['len']<=8192)

Filter:   0%|          | 0/16106 [00:00<?, ? examples/s]

In [32]:
ds_ftwctr_8192

Dataset({
    features: ['conversations', 'input_ids', 'len'],
    num_rows: 15970
})

In [33]:
ds_ftwctr_8192 = ds_ftwctr_8192.remove_columns(['input_ids', 'len'])

In [34]:
ds_ftwctr_8192.push_to_hub("habanoz/FineTume10k-WildChatTr6k-8192", private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


CommitInfo(commit_url='https://huggingface.co/datasets/habanoz/FineTume10k-WildChatTr6k-8192/commit/9b1702d0643a31baf64a9a8af57ae531f6bc4818', commit_message='Upload dataset', commit_description='', oid='9b1702d0643a31baf64a9a8af57ae531f6bc4818', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/habanoz/FineTume10k-WildChatTr6k-8192', endpoint='https://huggingface.co', repo_type='dataset', repo_id='habanoz/FineTume10k-WildChatTr6k-8192'), pr_revision=None, pr_num=None)